In [121]:
import pandas as pd
import DB
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
db = DB.MYDB()

Connect!


In [122]:
db.cursor.execute("Select r.gmkey, r.tcode, r.score, r.loss, m.tcodeA, m.tcodeH  FROM team_record_new r join game_meta_new m on r.gmkey = m.gmkey;")
data = pd.DataFrame(db.cursor.fetchall())
data

,gmkey,tcode,score,loss,tcodeA,tcodeH
0,S15G01N1,60,79,89,16,60
1,S15G01N1,16,89,79,16,60
2,S15G01N2,65,73,80,55,65
3,S15G01N2,55,80,73,55,65
4,S15G01N3,16,92,74,30,16
...,...,...,...,...,...,...
6901,S39G01N268,30,101,72,30,35
6902,S39G01N269,10,89,70,60,10
6903,S39G01N269,60,70,89,60,10
6904,S39G01N270,64,83,81,06,64


In [123]:
data['totalWin'] = 0
data['totalLose'] = 0
data['homeWin'] = 0
data['homeLose'] = 0
data['awayWin'] = 0
data['awayLose'] = 0
data['seasonWin'] = 0
data['seasonLose'] = 0
data['seasonHomeWin'] = 0
data['seasonHomeLose'] = 0
data['seasonAwayWin'] = 0
data['seasonAwayLose'] = 0

data['totalRelativeWin'] = 0
data['totalRelativeLose'] = 0
data['totalHomeRelativeWin'] = 0
data['totalHomeRelativeLose'] = 0
data['totalAwayRelativeWin'] = 0
data['totalAwayRelatvieLose'] = 0

data['seasonRelativeWin'] = 0
data['seasonRelativeLose'] = 0 
data['seasonHomeRelativeWin'] = 0
data['seasonHomeRelativeLose'] = 0
data['seasonAwayRelativeWin'] = 0
data['seasonAwayRelativeLose'] = 0
data

,gmkey,tcode,score,loss,tcodeA,tcodeH,totalWin,totalLose,homeWin,homeLose,...,totalHomeRelativeWin,totalHomeRelativeLose,totalAwayRelativeWin,totalAwayRelatvieLose,seasonRelativeWin,seasonRelativeLose,seasonHomeRelativeWin,seasonHomeRelativeLose,seasonAwayRelativeWin,seasonAwayRelativeLose
0,S15G01N1,60,79,89,16,60,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,S15G01N1,16,89,79,16,60,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,S15G01N2,65,73,80,55,65,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,S15G01N2,55,80,73,55,65,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,S15G01N3,16,92,74,30,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6901,S39G01N268,30,101,72,30,35,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6902,S39G01N269,10,89,70,60,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6903,S39G01N269,60,70,89,60,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6904,S39G01N270,64,83,81,06,64,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
db.cursor.execute("Select tcode  FROM team_code where category = '정규리그';")
tcodes = [code['tcode'] for code in db.cursor.fetchall()]
tcodes

['06', '10', '16', '30', '35', '50', '55', '60', '64', '65', '70']

In [125]:
from collections import defaultdict
seasonRecord = {str(i):defaultdict(dict) for i in range(15, 40, 2)}
for i in range(15, 40, 2):
    for tcodeA in tcodes:
        seasonRecord[str(i)][tcodeA] = {}
        for tcodeB in tcodes:
            if tcodeA == tcodeB:
                continue
            seasonRecord[str(i)][tcodeA][tcodeB] = {'win':0, 'lose':0}
            
totalRecord = {}
for tcodeA in tcodes:
    totalRecord[tcodeA] = {}
    for tcodeB in tcodes:
        if tcodeA == tcodeB:
            continue
        totalRecord[tcodeA][tcodeB] = {'win':0, 'lose':0}

In [126]:
for i in range(0, data.index.stop, 2):
    homeData = data.iloc[i].copy()
    awayData = data.iloc[i+1].copy()
    
    season = homeData['gmkey'][1:3]
    tcodeH = homeData['tcode']
    tcodeA = awayData['tcode']
    
    for t in tcodes:
        if t != tcodeH:
            #전체 승 / 패
            homeData['totalWin'] += totalRecord[tcodeH][t]['win'] + totalRecord[t][tcodeH]['lose']
            homeData['totalLose'] += totalRecord[tcodeH][t]['lose'] + totalRecord[t][tcodeH]['win']

            #전체 홈 승 / 패
            homeData['homeWin'] += totalRecord[tcodeH][t]['win']
            homeData['homeLose'] += totalRecord[tcodeH][t]['lose']

            #전체  어웨이 승 / 패
            homeData['awayWin'] += totalRecord[t][tcodeH]['lose']
            homeData['awayLose'] += totalRecord[t][tcodeH]['win']

            #시즌 승 / 패
            homeData['seasonWin'] += seasonRecord[season][tcodeH][t]['win'] + seasonRecord[season][t][tcodeH]['lose']
            homeData['seasonLose'] += seasonRecord[season][tcodeH][t]['lose'] + seasonRecord[season][t][tcodeH]['win']

            #시즌 홈 승 / 패
            homeData['seasonHomeWin'] += seasonRecord[season][tcodeH][t]['win']
            homeData['seasonHomeLose'] += seasonRecord[season][tcodeH][t]['lose']

            #시즌 어웨이 승 / 패
            homeData['seasonAwayWin'] += seasonRecord[season][t][tcodeH]['lose']
            homeData['seasonAwayLose'] += seasonRecord[season][t][tcodeH]['win']

        if t != tcodeA:
            #전체 승 / 패
            awayData['totalWin'] += totalRecord[tcodeA][t]['win'] + totalRecord[t][tcodeA]['lose']
            awayData['totalLose'] += totalRecord[tcodeA][t]['lose'] + totalRecord[t][tcodeA]['win']

            #전체 홈 승 / 패
            awayData['homeWin'] += totalRecord[tcodeA][t]['win']
            awayData['homeLose'] += totalRecord[tcodeA][t]['lose']

            #전체 어웨이 승 / 패
            awayData['awayWin'] += totalRecord[t][tcodeA]['lose']
            awayData['awayLose'] += totalRecord[t][tcodeA]['win']

            #시즌 승 / 패
            awayData['seasonWin'] += seasonRecord[season][tcodeA][t]['win'] + seasonRecord[season][t][tcodeA]['lose']
            awayData['seasonLose'] += seasonRecord[season][tcodeA][t]['lose'] + seasonRecord[season][t][tcodeA]['win']

            #시즌 홈 승 / 패
            awayData['seasonHomeWin'] += seasonRecord[season][tcodeA][t]['win']
            awayData['seasonHomeLose'] += seasonRecord[season][tcodeA][t]['lose']

            #시즌 어웨이 승 / 패
            awayData['seasonAwayWin'] += seasonRecord[season][t][tcodeA]['lose']
            awayData['seasonAwayLose'] += seasonRecord[season][t][tcodeA]['win']

    
    #역대 상대 전적
    homeData['totalRelativeWin'] = totalRecord[tcodeH][tcodeA]['win'] + totalRecord[tcodeA][tcodeH]['lose']
    homeData['totalRelativeLose'] = totalRecord[tcodeH][tcodeA]['lose'] + totalRecord[tcodeA][tcodeH]['win']

    awayData['totalRelativeWin'] = totalRecord[tcodeA][tcodeH]['win'] + totalRecord[tcodeH][tcodeA]['lose']
    awayData['totalRelativeLose'] = totalRecord[tcodeA][tcodeH]['lose'] + totalRecord[tcodeH][tcodeA]['win']

    #역대 홈 상대 전적
    homeData['totalHomeRelativeWin'] = totalRecord[tcodeH][tcodeA]['win']
    homeData['totalHomeRelativeLose'] = totalRecord[tcodeH][tcodeA]['lose']

    awayData['totalHomeRelativeWin'] = totalRecord[tcodeA][tcodeH]['win']
    awayData['totalHomeRelativeLose'] = totalRecord[tcodeA][tcodeH]['lose']

    #역대 어웨이 상대 전적
    homeData['totalAwayRelativeWin'] = totalRecord[tcodeA][tcodeH]['lose']
    homeData['totalAwayRelatvieLose'] = totalRecord[tcodeA][tcodeH]['win']

    awayData['totalAwayRelativeWin'] = totalRecord[tcodeH][tcodeA]['lose']
    awayData['totalAwayRelatvieLose'] = totalRecord[tcodeH][tcodeA]['win']


    #시즌 상대 전적
    homeData['seasonRelativeWin'] = seasonRecord[season][tcodeH][tcodeA]['win'] + seasonRecord[season][tcodeA][tcodeH]['lose']
    homeData['seasonRelativeLose'] = seasonRecord[season][tcodeH][tcodeA]['lose'] + seasonRecord[season][tcodeA][tcodeH]['win']

    awayData['seasonRelativeWin'] = seasonRecord[season][tcodeA][tcodeH]['win'] + seasonRecord[season][tcodeH][tcodeA]['lose']
    awayData['seasonRelativeLose'] = seasonRecord[season][tcodeA][tcodeH]['lose'] + seasonRecord[season][tcodeH][tcodeA]['win']

    #시즌 홈 상대 전적
    homeData['seasonHomeRelativeWin'] = seasonRecord[season][tcodeH][tcodeA]['win'] 
    homeData['seasonHomeRelativeLose'] = seasonRecord[season][tcodeH][tcodeA]['lose'] 

    awayData['seasonHomeRelativeWin'] = seasonRecord[season][tcodeA][tcodeH]['win']
    awayData['seasonHomeRelativeLose'] = seasonRecord[season][tcodeA][tcodeH]['lose']

    # 시즌 어웨이 상대 전적
    homeData['seasonAwayRelativeWin'] = seasonRecord[season][tcodeA][tcodeH]['lose']
    homeData['seasonAwayRelativeLose'] = seasonRecord[season][tcodeA][tcodeH]['win']

    awayData['seasonAwayRelativeWin'] = seasonRecord[season][tcodeH][tcodeA]['lose']
    awayData['seasonAwayRelativeLose'] = seasonRecord[season][tcodeH][tcodeA]['win']

    data.iloc[i] = homeData
    data.iloc[i+1] = awayData
    # 기록 업데이트
    # 홈 팀이 이긴 경우
    if homeData['score'] > awayData['score']:
        totalRecord[tcodeH][tcodeA]['win'] += 1
        seasonRecord[season][tcodeH][tcodeA]['win'] += 1
    
    elif homeData['score'] < awayData['score']:
        totalRecord[tcodeH][tcodeA]['lose'] += 1
        seasonRecord[season][tcodeH][tcodeA]['lose'] += 1

    
    print(i, '번째 데이터 수정 완료')
    

0 번째 데이터 수정 완료
2 번째 데이터 수정 완료
4 번째 데이터 수정 완료
6 번째 데이터 수정 완료
8 번째 데이터 수정 완료
10 번째 데이터 수정 완료
12 번째 데이터 수정 완료
14 번째 데이터 수정 완료
16 번째 데이터 수정 완료
18 번째 데이터 수정 완료
20 번째 데이터 수정 완료
22 번째 데이터 수정 완료
24 번째 데이터 수정 완료
26 번째 데이터 수정 완료
28 번째 데이터 수정 완료
30 번째 데이터 수정 완료
32 번째 데이터 수정 완료
34 번째 데이터 수정 완료
36 번째 데이터 수정 완료
38 번째 데이터 수정 완료
40 번째 데이터 수정 완료
42 번째 데이터 수정 완료
44 번째 데이터 수정 완료
46 번째 데이터 수정 완료
48 번째 데이터 수정 완료
50 번째 데이터 수정 완료
52 번째 데이터 수정 완료
54 번째 데이터 수정 완료
56 번째 데이터 수정 완료
58 번째 데이터 수정 완료
60 번째 데이터 수정 완료
62 번째 데이터 수정 완료
64 번째 데이터 수정 완료
66 번째 데이터 수정 완료
68 번째 데이터 수정 완료
70 번째 데이터 수정 완료
72 번째 데이터 수정 완료
74 번째 데이터 수정 완료
76 번째 데이터 수정 완료
78 번째 데이터 수정 완료
80 번째 데이터 수정 완료
82 번째 데이터 수정 완료
84 번째 데이터 수정 완료
86 번째 데이터 수정 완료
88 번째 데이터 수정 완료
90 번째 데이터 수정 완료
92 번째 데이터 수정 완료
94 번째 데이터 수정 완료
96 번째 데이터 수정 완료
98 번째 데이터 수정 완료
100 번째 데이터 수정 완료
102 번째 데이터 수정 완료
104 번째 데이터 수정 완료
106 번째 데이터 수정 완료
108 번째 데이터 수정 완료
110 번째 데이터 수정 완료
112 번째 데이터 수정 완료
114 번째 데이터 수정 완료
116 번째 데이터 수정 완료
118 번째 데이터 수정 완료
120 번째 데이터 수정 완료
122 번째 데이터 수정 완료
1

In [127]:
result = data.drop(columns=['score', 'loss', 'tcodeA', 'tcodeH'])
result.to_csv('win_lose_record.csv')

In [128]:
result.index += 1
result

,gmkey,tcode,totalWin,totalLose,homeWin,homeLose,awayWin,awayLose,seasonWin,seasonLose,...,totalHomeRelativeWin,totalHomeRelativeLose,totalAwayRelativeWin,totalAwayRelatvieLose,seasonRelativeWin,seasonRelativeLose,seasonHomeRelativeWin,seasonHomeRelativeLose,seasonAwayRelativeWin,seasonAwayRelativeLose
1,S15G01N1,60,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,S15G01N1,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,S15G01N2,65,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,S15G01N2,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,S15G01N3,16,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,S39G01N268,30,316,374,173,175,143,199,26,27,...,26,12,19,18,4,1,2,1,2,0
6903,S39G01N269,10,428,261,218,125,210,136,29,24,...,20,17,21,17,1,4,0,2,1,2
6904,S39G01N269,60,341,348,189,155,152,193,21,32,...,17,21,17,20,4,1,2,1,2,0
6905,S39G01N270,64,26,27,13,13,13,14,26,27,...,0,2,1,2,1,4,0,2,1,2


In [129]:
myDB = DB.HYGPDB()
result.to_sql(name='win_lose_record', con=myDB.engine, if_exists='append', index=True, index_label='win_lose_record_idx', chunksize=1000)

Exception during reset or similar
Traceback (most recent call last):
  File "/Users/grulsuitg/.conda/envs/KBLPredict/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/grulsuitg/.conda/envs/KBLPredict/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/grulsuitg/.conda/envs/KBLPredict/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
  File "/Users/grulsuitg/.conda/envs/KBLPredict/lib/python3.10/site-packages/pymysql/connections.py", line 480, in rollback
    self._read_ok_packet()
  File "/Users/grulsuitg/.conda/envs/KBLPredict/lib/python3.10/site-packages/pymysql/connections.py", line 443, in _read_ok_packet
    pkt = self._read_packet()
  File "/Users/grulsuitg/.conda/envs/KBLPredict/lib/python3.10/site-packages/pymysql/connections.py", line 692, in _read_packet
    packet_h

6906